In [1]:
import string 
import math
import numpy as np 
import pandas as pd 
pd.set_option('display.max_columns', 100) 
pd.set_option('display.max_rows', 10) 
pd.set_option('display.width', 1000) 
from itertools import cycle 

In [2]:
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC, LinearSVC
from sklearn.ensemble import RandomForestRegressor
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.linear_model import Perceptron
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier
import sklearn
import datetime
####
# scikit-learn version is 0.24.1.
# pandas version is 1.2.3.
# numpy version is 1.20.1.

In [3]:
df=pd.read_csv(r'D:\учеба\Kaggle\Биржа\US1.AAPL_200501_210901.csv' ,sep=',')

In [4]:
#фичи delta1 - разница <CLOSE> с предыдущим временным промежутком,
# delta2 - с точкой двумя промежутками раньше
delta1=[0 for i in range(0,df["<TICKER>"].size)]
delta2=[0 for i in range(0,df["<TICKER>"].size)]
i=1
while (i<df["<TICKER>"].size):
    delta1[i]=df["<CLOSE>"][i]-df["<CLOSE>"][i-1]
    i+=1
df['delta1']=delta1
delta1=[0 for i in range(0,df["<TICKER>"].size)]

i=2
while (i<df["<TICKER>"].size):
    delta2[i]=df["<CLOSE>"][i]-df["<CLOSE>"][i-2]
    i+=1
df['delta2']=delta2
df

,<TICKER>,<PER>,<DATE>,<TIME>,<OPEN>,<HIGH>,<LOW>,<CLOSE>,<VOL>,delta1,delta2
0,US1.AAPL,1,01/05/20,16:31,286.220,289.310,285.850,289.29,64648,0.00,0.00
1,US1.AAPL,1,01/05/20,16:32,289.250,289.350,288.470,289.04,33806,-0.25,0.00
2,US1.AAPL,1,01/05/20,16:33,289.160,289.720,288.340,288.56,33746,-0.48,-0.73
3,US1.AAPL,1,01/05/20,16:34,288.300,289.380,288.300,289.16,22446,0.60,0.12
4,US1.AAPL,1,01/05/20,16:35,289.170,290.440,288.960,290.44,26771,1.28,1.88
...,...,...,...,...,...,...,...,...,...,...,...
131467,US1.AAPL,1,01/09/21,22:56,152.430,152.455,152.390,152.44,48179,0.02,-0.01
131468,US1.AAPL,1,01/09/21,22:57,152.445,152.470,152.420,152.44,30555,0.00,0.02
131469,US1.AAPL,1,01/09/21,22:58,152.445,152.470,152.410,152.45,29922,0.01,0.01
131470,US1.AAPL,1,01/09/21,22:59,152.460,152.495,152.440,152.49,43008,0.04,0.05


In [5]:
#добавляю фичу "день недели" + форматирую дату с дд\мм\гг на дд\мм\гггг
day=[0 for i in range(0,df["<TICKER>"].size)]
from datetime import datetime, date, time
i=0
while (i<df["<TICKER>"].size):
    df["<DATE>"][i]=df["<DATE>"][i][0:6]+"20"+df["<DATE>"][i][6:8]
    day[i]=datetime.weekday(datetime.strptime(df["<DATE>"][i]+" "+df["<TIME>"][i], 
                             "%d/%m/%Y %H:%M"))
    i+=1
df['day']=day

<ipython-input-5-314177c4a164>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["<DATE>"][i]=df["<DATE>"][i][0:6]+"20"+df["<DATE>"][i][6:8]


In [6]:
#добавляю фичу "время" - в секундах с начала торгов (16:30)
size=len(df["<TICKER>"])
hours=[0 for i in range(size)]
mins=[0 for i in range(size)]
secs=[0 for i in range(size)]
for i in range(size):
    hours[i]=int(df["<TIME>"][i][0:2])
    mins[i]=int(df["<TIME>"][i][3:5])
TIME=[0 for i in range(0,df["<TICKER>"].size)]
for i in range(size):
    TIME[i]=(hours[i]-16)*3600+mins[i]*60-1860
    
df['time']=TIME

In [7]:
#метки для обучения (значения <CLOSE> на следующем временном промежутке)
labels=[float(0.0) for i in range(0,df["<TICKER>"].size)]
labels=np.float32(labels)
df['label']=labels
for i in range(size-1):
    df['label'][i]=(df['<CLOSE>'][i+1])

<ipython-input-7-99c929464daa>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['label'][i]=(df['<CLOSE>'][i+1])


In [8]:
#формирование обучающей, тестовой выборок
train={
    'time':df['time'][0:size-1],
    '<CLOSE>':df['<CLOSE>'][0:size-1],
      'delta1':df['delta1'][0:size-1],
    'delta2':df['delta2'][0:size-1],
'day':df['day'][0:size-5]}
df_train=pd.DataFrame(data=train)
X = df_train

y=df['label'][0:size-1]

from sklearn.model_selection import train_test_split

In [9]:
df_train.values

array([[ 0.0000e+00,  2.8929e+02,  0.0000e+00,  0.0000e+00,  4.0000e+00],
       [ 6.0000e+01,  2.8904e+02, -2.5000e-01,  0.0000e+00,  4.0000e+00],
       [ 1.2000e+02,  2.8856e+02, -4.8000e-01, -7.3000e-01,  4.0000e+00],
       ...,
       [ 2.3160e+04,  1.5244e+02,  0.0000e+00,  2.0000e-02,         nan],
       [ 2.3220e+04,  1.5245e+02,  1.0000e-02,  1.0000e-02,         nan],
       [ 2.3280e+04,  1.5249e+02,  4.0000e-02,  5.0000e-02,         nan]])

In [10]:
y=df['label'][0:size-1]

In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state =0)


In [12]:
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

In [13]:
X_train

,time,<CLOSE>,delta1,delta2,day
87224,12840,123.18,-0.02,-0.05,1.0
71422,4440,144.46,0.32,0.20,0.0
108857,120,126.21,0.04,-0.06,4.0
75866,13680,136.69,0.04,0.04,1.0
88232,3060,119.97,0.00,0.11,4.0
...,...,...,...,...,...
41993,15660,113.35,-0.02,-0.09,4.0
97639,5700,133.27,-0.02,-0.07,4.0
95939,20760,134.82,0.09,0.06,4.0
117952,7800,148.75,-0.03,-0.09,3.0


In [14]:
y_test=y_test.reset_index(drop=True)

In [15]:
# обучение
count=5
mean_square_err=[0]*count
deviation=[0]*count
mae=[0]*count
for depth in range(count):
    regr = RandomForestRegressor(max_depth=5+depth, random_state=0,min_samples_leaf=5,criterion='mse')
    regr.fit(X_train,y_train)
    y_pred=regr.predict(X_test)
    for i in range(y_pred.size):
        mean_square_err[depth]+=(y_test[i]-y_pred[i])**2
    mean_square_err[depth]/=y_pred.size
    mean_square_err[depth]=mean_square_err[depth]**0.5
    for i in range(y_pred.size):
        deviation[depth]+=abs(y_test[i]-y_pred[i])*100/y_test[i]
    deviation[depth]/=y_pred.size
    for i in range(y_pred.size):
        mae[depth]+=abs(y_test[i]-y_pred[i])
    mae[depth]/=y_pred.size

In [16]:
y_pred=regr.predict(X_test)
X_test, y_pred

(         time  <CLOSE>  delta1  delta2  day
 101075   1260   124.52    0.07    0.04  3.0
 122139   1560   144.91   -0.04   -0.12  4.0
 112446   4860   134.15    0.06   -0.01  3.0
 52927   19860   119.21    0.02    0.09  2.0
 123770   5820   147.69   -0.01   -0.05  3.0
 ...       ...      ...     ...     ...  ...
 54661    6900   119.08    0.10    0.11  2.0
 104628   3780   127.00    0.00    0.01  2.0
 101126   4320   125.52   -0.04   -0.16  3.0
 58728    4320   122.28    0.20    0.05  4.0
 3459    20340   304.74   -0.02   -0.08  2.0
 
 [26295 rows x 5 columns],
 array([124.50600418, 144.94489486, 134.14867434, ..., 125.54351921,
        122.26760486, 304.6302384 ]))

In [17]:
err=pd.DataFrame(data={'mse':mean_square_err,'mae':mae,'dev':deviation})

In [18]:
err

,mse,mae,dev
0,1.761382,1.129557,0.543780
1,0.894492,0.554460,0.265342
2,0.457351,0.270360,0.128683
3,0.277351,0.151869,0.074510
4,0.235301,0.119734,0.061101


In [19]:
# итоговая модель:
model = RandomForestRegressor(max_depth=8, random_state=0,min_samples_leaf=5)
model.fit(X_train,y_train)

RandomForestRegressor(max_depth=8, min_samples_leaf=5, random_state=0)

In [20]:
model.feature_importances_

array([2.03054748e-05, 9.99979679e-01, 6.17605127e-09, 5.29529439e-09,
       3.83961057e-09])

In [21]:
#Модель нейронной сети PyTorch

import torch
import torch.nn as nn
import torch.nn.functional as F
import torchvision

min_max_scaler = sklearn.preprocessing.MinMaxScaler()

train_data=(torch.tensor(min_max_scaler.fit_transform (X_train.values)))
train_target=(torch.tensor( (y_train.values)))
test_data=(torch.tensor(min_max_scaler.fit_transform (X_test.values)))
test_target=(torch.tensor( (y_test.values)))

In [36]:
class Net(nn.Module):
    def __init__(self,drop):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(5, 200)
        self.fc2 = nn.Linear(200, 200)
        self.fc3 = nn.Linear(200, 100)
        self.fc4 = nn.Linear(100, 1)
        self.dropout = nn.Dropout(drop)
    def forward(self, x):
        #x = F.logsigmoid(self.fc1(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = F.relu(self.fc3(x))
        x = self.dropout(x)
        x = self.fc4(x)
        return x

In [38]:
#обучение модели и возврат mae

def checkloss(drop=0.1,learning_rate=0.01, batch_size=30):
    epochs=10
    train_batches=math.ceil(X_train["time"].size/batch_size)-1
    net = Net(drop)
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    #criterion = nn.MSELoss()
    criterion=nn.L1Loss()
    for epoch in range(epochs):
        for batch in range(train_batches):
            optimizer.zero_grad()
            net_out=net(train_data[batch*batch_size:(batch_size*(batch+1))].float())
            loss = criterion(net_out, train_target[batch*batch_size:(batch_size*(batch+1))].float())
            loss.backward()
            optimizer.step()
    mae=0
    net_out=net(test_data.float())
    for i in range(test_data.shape[0]):
        mae+=abs(net_out[i]-test_target[i])
    mae/=test_data.shape[0]
    mae
    return mae

In [39]:
#Обучение модели и ее возврат

def fit(drop=0.1,learning_rate=0.01, batch_size=30):
    epochs=10
    train_batches=math.ceil(X_train["time"].size/batch_size)-1
    net = Net(drop)
    optimizer = torch.optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9)
    #criterion = nn.MSELoss()
    criterion=nn.L1Loss()
    for epoch in range(epochs):
        for batch in range(train_batches):
            optimizer.zero_grad()
            net_out=net(train_data[batch*batch_size:(batch_size*(batch+1))].float())
            loss = criterion(net_out, train_target[batch*batch_size:(batch_size*(batch+1))].float())
            loss.backward()
            optimizer.step()
    return net

In [37]:
#grid search

search=pd.DataFrame(columns=["drop","lr","batch","loss"])
count=0
for i in np.arange(0.0,0.51,0.05):
    for j in np.arange(0.001,0.01,0.001):
        for k in np.arange(20,71,5):
            search.loc[count]=[i,j,k,checkloss(i,j,k).detach()]
            count+=1
search

KeyboardInterrupt: 

In [ ]:
gsmin=search["loss"].min()
minid=0
for i in range(search["drop"].size):
    if (search["loss"][i]==gsmin):
        minid=i
search.loc[minid]
#drop=0, lr=0.006, batch=20, mae=66.7938

In [ ]:
net=fit(0,0.006,20)

In [ ]:
#Итоговая nn модель
net